In [22]:
import math

import matplotlib as mpl
from matplotlib import cm
mpl.rcParams['text.usetex'] = True 
mpl.rcParams['text.latex.preamble'] = [r'\usepackage{libertine}', r'\usepackage{newtxmath}'] 
mpl.rc('font', family='serif')

import numpy as np
import pandas as pd
import seaborn as sns
import copy

import matplotlib.pyplot as plt
from matplotlib import rc

from cycler import cycler

from scipy.interpolate import make_interp_spline, BSpline

mpl.rcParams['ps.usedistiller'] = 'xpdf' 

plt.style.use('seaborn-notebook')

plt.rcParams['axes.titlesize'] = '25'
plt.rcParams['axes.labelsize'] = '25'
plt.rcParams['legend.fontsize'] = '15'
plt.rcParams['xtick.labelsize'] = '18'
plt.rcParams['ytick.labelsize'] = '18'

monochrome=(cycler('color', sns.color_palette("husl", 8))*2+(cycler('marker', ['v', 's', "o"])*7)[0:16])
plt.rc('axes', prop_cycle=monochrome)

pd.options.display.max_rows = 999
pd.options.display.max_columns = None
cmap = cm.get_cmap('RdBu')

husl4 = sns.color_palette("husl", 4)
husl5 = sns.color_palette("husl", 5)
husl6 = sns.color_palette("husl", 7)
greens5 = sns.color_palette("Greens_r", 5)
purples5 = sns.color_palette("Purples_r", 7)
blues5 = sns.color_palette("Blues_r", 5)
reds5 = sns.color_palette("Reds_r", 8)
oranges5 = sns.color_palette("Oranges_r", 5)
greys5 = sns.color_palette("Greys_r", 5)
winter5 = sns.color_palette("viridis_r", 5)

In [23]:
folder = "2020-04-28-09-50_SGMRDminers_" # Replace

In [24]:
summary = pd.read_csv("../experiments/" + folder + "/SGMRDminers.csv")

In [25]:
searchers = []
for x in summary["subspaces"]:
    if("None" in x): 
        searchers += ["GMD-Init"]
    if("-RD" in x): 
        searchers += ["SGMRD-RD"]
    if("Worst" in x): 
        searchers += ["SGMRD-GD"]
    if("MPTS" in x): 
        searchers += ["SGMRD-MPTS"]
    if("All" in x): 
        searchers += ["GMD-Batch"]
    elif("Full" in x): 
        searchers += ["Full"]
len(searchers) == len(summary["subspaces"])

True

In [26]:
summary["searchers"] = searchers

In [27]:
summary["searchers"].unique()

array(['GMD-Init', 'SGMRD-RD', 'SGMRD-GD', 'SGMRD-MPTS', 'GMD-Batch',
       'Full'], dtype=object)

In [28]:
summary["minerId"].unique()

array(['Hash', 'LOF-1', 'LOF-2', 'LOF-5', 'LOF-10', 'LOF-20', 'LOF-50',
       'LOF-100'], dtype=object)

In [29]:
summary["refId"].unique()

array(['activity', 'kdd99', 'example_10', 'example_20', 'example_50'], dtype=object)

In [30]:
toexclude = ["SGMRD-GD", "SGMRD-RD"]
summary = summary[[x not in toexclude for x in summary["searchers"]]]

In [31]:
summaryhash = summary[["Hash" in x for x in summary["minerId"]]]

In [32]:
summaryfull = summary[[x == "Full" for x in summary["searchers"]]]
summary = summary[[x != "Full" for x in summary["searchers"]]]

In [33]:
summary.groupby(["refId", "minerId", "searchers"]).mean()

minetime  rocauc   prauc      ap      p1  \
refId      minerId searchers                                                 
activity   Hash    GMD-Batch    28952.8656  0.9670     NaN  0.7989  0.9820   
                   GMD-Init     32239.5874  0.9718     NaN  0.8375  1.0000   
                   SGMRD-MPTS   29557.1857  0.9706     NaN  0.8553  1.0000   
           LOF-1   GMD-Batch   153722.1203  0.7542     NaN  0.2073  0.1261   
                   GMD-Init    167168.9962  0.7711     NaN  0.2207  0.1351   
                   SGMRD-MPTS  158119.0237  0.7549     NaN  0.2061  0.1532   
           LOF-10  GMD-Batch   160306.7702  0.9679     NaN  0.8120  0.9775   
                   GMD-Init    192178.8364  0.9686     NaN  0.8164  0.9865   
                   SGMRD-MPTS  181439.3420  0.9718     NaN  0.8238  0.9685   
           LOF-100 GMD-Batch   168160.9243  0.9489     NaN  0.6156  0.6757   
                   GMD-Init    371399.4057  0.9269     NaN  0.5461  0.6081   
                   SGMRD-MPTS  176792.0153  0.9509     NaN  0.6301  0.7297   
           LOF-2   GMD-Batch   162144.1978  0.9122     NaN  0.5880  0.7297   
                   GMD-Init    170137.8035  0.9281     NaN  0.6670  0.8604   
                   SGMRD-MPTS  156345.0517  0.9199     NaN  0.5960  0.6757   
           LOF-20  GMD-Batch   129312.4354  0.9491     NaN  0.6479  0.8964   
                   GMD-Init    210970.8246  0.9465     NaN  0.6415  0.9234   
                   SGMRD-MPTS  198768.5232  0.9545     NaN  0.6768  0.9640   
           LOF-5   GMD-Batch   157907.5067  0.9664     NaN  0.8421  0.9820   
                   GMD-Init    179685.4663  0.9723     NaN  0.8656  1.0000   
                   SGMRD-MPTS  168964.7974  0.9732     NaN  0.8539  0.9459   
           LOF-50  GMD-Batch   135510.9959  0.9499     NaN  0.5870  0.6036   
                   GMD-Init    275159.7327  0.9394     NaN  0.5491  0.5676   
                   SGMRD-MPTS  194216.2111  0.9519     NaN  0.6080  0.7072   
example_10 Hash    GMD-Batch     2376.3763  0.8351  0.1694  0.1694  0.2000   
                   GMD-Init      2347.9227  0.8324  0.1404  0.1404  0.2200   
                   SGMRD-MPTS    2369.3498  0.8257  0.1663  0.1663  0.2200   
           LOF-1   GMD-Batch    10409.0614  0.5840  0.0305  0.0305  0.0600   
                   GMD-Init     11400.6883  0.6050  0.0379  0.0379  0.0700   
                   SGMRD-MPTS    9664.0238  0.6225  0.0284  0.0284  0.0700   
           LOF-10  GMD-Batch    11836.1124  0.8081  0.6130  0.6130  0.5300   
                   GMD-Init     13154.0920  0.8310  0.5853  0.5853  0.4900   
                   SGMRD-MPTS   11570.4355  0.8137  0.6224  0.6224  0.5300   
           LOF-100 GMD-Batch    13617.4237  0.9057  0.5566  0.5566  0.4500   
                   GMD-Init     26358.7080  0.8943  0.5185  0.5185  0.4400   
                   SGMRD-MPTS   14244.9622  0.9270  0.5993  0.5993  0.5000   
           LOF-2   GMD-Batch    10532.5114  0.7130  0.2707  0.2707  0.2800   
                   GMD-Init     11584.6998  0.7462  0.2775  0.2775  0.2800   
                   SGMRD-MPTS   11127.3290  0.7839  0.3034  0.3034  0.2900   
           LOF-20  GMD-Batch     9664.7777  0.8227  0.5966  0.5966  0.5000   
                   GMD-Init     14765.5255  0.8463  0.5831  0.5831  0.4800   
                   SGMRD-MPTS   13743.4890  0.8360  0.6132  0.6132  0.5200   
           LOF-5   GMD-Batch     8824.1417  0.7905  0.5760  0.5760  0.4900   
                   GMD-Init     12260.1755  0.8132  0.5640  0.5640  0.4800   
                   SGMRD-MPTS    9933.1688  0.7941  0.5997  0.5997  0.5100   
           LOF-50  GMD-Batch    10380.0255  0.8601  0.5742  0.5742  0.4700   
                   GMD-Init     18832.7246  0.8707  0.5673  0.5673  0.4700   
                   SGMRD-MPTS   17406.6255  0.8756  0.6052  0.6052  0.5100   
example_20 Hash    GMD-Batch     5039.3794  0.7869  0.0598  0.0598  0.1100   
                   GMD-Init      5538.0362  0.6725  0.01

In [34]:
summarylof = summary[["LOF" in x for x in summary["minerId"]]]

In [35]:
summaryloffull = summaryfull[["LOF" in x for x in summaryfull["minerId"]]]

In [36]:
summarylof.groupby(["refId", "searchers"])["rocauc"].idxmax()

refId       searchers 
activity    GMD-Batch      33
            GMD-Init        4
            SGMRD-MPTS     26
example_10  GMD-Batch     135
            GMD-Init      105
            SGMRD-MPTS    133
example_20  GMD-Batch     183
            GMD-Init      149
            SGMRD-MPTS    182
example_50  GMD-Batch     225
            GMD-Init      202
            SGMRD-MPTS    220
kdd99       GMD-Batch      80
            GMD-Init       50
            SGMRD-MPTS     73
Name: rocauc, dtype: int64

In [37]:
interesting = ["minetime", "rocauc", "ap", "p1", "p2", "p5", "r1", "r2", "r5"]

In [38]:
summarylof.loc[summarylof.groupby(["refId", "searchers"])["rocauc"].idxmax()].groupby(["refId", "minerId", "searchers"])[interesting].mean()*100

minetime  rocauc     ap      p1     p2  \
refId      minerId searchers                                               
activity   LOF-10  GMD-Batch   16030677.02   96.79  81.20   97.75  96.18   
           LOF-5   GMD-Init    17968546.63   97.23  86.56  100.00  98.65   
                   SGMRD-MPTS  16896479.74   97.32  85.39   94.59  94.83   
example_10 LOF-100 GMD-Batch    1361742.37   90.57  55.66   45.00  25.00   
                   GMD-Init     2635870.80   89.43  51.85   44.00  24.50   
                   SGMRD-MPTS   1424496.22   92.70  59.93   50.00  26.00   
example_20 LOF-100 GMD-Batch    3261954.86   83.93  39.07   35.00  20.00   
                   SGMRD-MPTS   3441590.82   85.05  41.19   36.00  19.50   
           LOF-5   GMD-Init     3738080.91   78.40  27.91   24.00  14.00   
example_50 LOF-5   GMD-Batch    9691720.29   78.31  30.90   27.00  15.50   
                   SGMRD-MPTS  11152072.57   75.87  31.27   27.00  16.00   
           LOF-50  GMD-Init    22845428.92   62.52   7.28   10.00   6.00   
kdd99      LOF-1   GMD-Batch   12104868.47   68.74  10.01    0.00   0.20   
                   GMD-Init    12611218.13   73.31  11.38    0.00   0.80   
                   SGMRD-MPTS  11394691.12   69.98  10.29    0.00   0.20   

                                  p5     r1     r2     r5  
refId      minerId searchers                               
activity   LOF-10  GMD-Batch   88.49   9.75  19.24  44.22  
           LOF-5   GMD-Init    94.87   9.98  19.73  47.42  
                   SGMRD-MPTS  94.24   9.44  18.97  47.10  
example_10 LOF-100 GMD-Batch   12.00  52.33  58.14  69.77  
                   GMD-Init    10.80  51.16  56.98  62.79  
                   SGMRD-MPTS  12.00  58.14  60.47  69.77  
example_20 LOF-100 GMD-Batch    8.80  39.77  45.45  50.00  
                   SGMRD-MPTS   9.20  40.91  44.32  52.27  
           LOF-5   GMD-Init     7.60  27.27  31.82  43.18  
example_50 LOF-5   GMD-Batch    7.40  33.33  38.27  45.68  
                   SGMRD-MPTS   7.60  33.33  39.51  46.91  
           LOF-50  GMD-Init     3.40  12.35  14.81  20.99  
kdd99      LOF-1   GMD-Batch    1.84   0.00   0.06   1.29  
                   GMD-Init     3.84   0.00   0.22   2.70  
                   SGMRD-MPTS   0.56   0.00   0.06   0.39

In [39]:
summaryloffull.loc[summaryloffull.groupby(["refId", "searchers"])["rocauc"].idxmax()].groupby(["refId", "minerId", "searchers"])[interesting].mean()*100

,,,minetime,rocauc,ap,p1,p2,p5,r1,r2,r5
refId,minerId,searchers,,,,,,,,,
activity,LOF-50,Full,4212001.29,93.93,61.80,74.32,64.72,64.03,7.42,12.94,32.00
example_10,LOF-5,Full,675721.34,88.77,31.44,33.00,18.50,10.40,38.37,43.02,60.47
example_20,LOF-2,Full,1367652.88,72.55,5.57,8.00,6.00,4.40,9.09,13.64,25.00
example_50,LOF-1,Full,2824681.33,61.38,1.08,0.00,0.50,0.60,0.00,1.23,3.70
kdd99,LOF-1,Full,1578129.80,70.37,10.79,0.00,0.00,0.08,0.00,0.00,0.06


In [40]:
summaryhash.groupby(["refId", "minerId", "searchers"])[interesting].mean()*100

minetime  rocauc     ap      p1     p2  \
refId      minerId searchers                                              
activity   Hash    Full         227695.12   95.95  68.23   71.62  72.58   
                   GMD-Batch   2895286.56   96.70  79.89   98.20  97.08   
                   GMD-Init    3223958.74   97.18  83.75  100.00  98.20   
                   SGMRD-MPTS  2955718.57   97.06  85.53  100.00  99.33   
example_10 Hash    Full          44686.34   71.23   1.87    0.00   0.00   
                   GMD-Batch    237637.63   83.51  16.94   20.00  16.00   
                   GMD-Init     234792.27   83.24  14.04   22.00  12.00   
                   SGMRD-MPTS   236934.98   82.57  16.63   22.00  12.50   
example_20 Hash    Full          59970.21   48.39   0.80    0.00   0.00   
                   GMD-Batch    503937.94   78.69   5.98   11.00   9.00   
                   GMD-Init     553803.62   67.25   1.58    0.00   0.50   
                   SGMRD-MPTS   521609.86   75.42   6.37   10.00   8.00   
example_50 Hash    Full         103618.53   46.52   0.73    0.00   0.00   
                   GMD-Batch   1490473.65   52.73   0.83    0.00   0.00   
                   GMD-Init    1569148.49   58.61   1.20    0.00   0.50   
                   SGMRD-MPTS  1427436.82   54.99   0.94    0.00   0.00   
kdd99      Hash    Full         194963.46   26.58   4.50    2.40   2.40   
                   GMD-Batch   3037476.81   43.21   5.73    0.00   0.00   
                   GMD-Init    2973735.13   53.29   7.03    0.40   0.20   
                   SGMRD-MPTS  2994414.32   44.19   5.95    0.00   0.40   

                                  p5     r1     r2     r5  
refId      minerId searchers                               
activity   Hash    Full        75.00   7.15  14.52  37.48  
                   GMD-Batch   88.04   9.80  19.42  44.00  
                   GMD-Init    92.45   9.98  19.64  46.20  
                   SGMRD-MPTS  94.60   9.98  19.87  47.28  
example_10 Hash    Full         2.80   0.00   0.00  16.28  
                   GMD-Batch    8.20  23.26  37.21  47.67  
                   GMD-Init     7.20  25.58  27.91  41.86  
                   SGMRD-MPTS   8.00  25.58  29.07  46.51  
example_20 Hash    Full         0.00   0.00   0.00   0.00  
                   GMD-Batch    6.00  12.50  20.45  34.09  
                   GMD-Init     1.80   0.00   1.14  10.23  
                   SGMRD-MPTS   3.80  11.36  18.18  21.59  
example_50 Hash    Full         0.00   0.00   0.00   0.00  
                   GMD-Batch    0.20   0.00   0.00   1.23  
                   GMD-Init     1.80   0.00   1.23  11.11  
                   SGMRD-MPTS   1.20   0.00   0.00   7.41  
kdd99      Hash    Full         1.44   0.34   0.67   1.01  
                   GMD-Batch    0.08   0.00   0.00   0.06  
                   GMD-Init     0.24   0.06   0.06   0.17  
                   SGMRD-MPTS   0.16   0.00   0.11   0.11

In [41]:
summary[(summary["refId"] == "kdd99") & (summary["searchers"] == "SGMRD-MPTS")]

,refId,minerId,subspaces,minetime,rocauc,prauc,ap,p1,p2,p5,p10,p20,p30,r1,r2,r5,r10,r20,r30,searchers
58,kdd99,Hash,/home/edouardfouche/data/subspaces/kdd99-SGMRD-KSP...,29944.1432,0.4419,0.0607,0.0595,0.0,0.004,0.0016,0.0016,0.0116,0.0660,0.0,0.0011,0.0011,0.0022,0.0326,0.2782,SGMRD-MPTS
73,kdd99,LOF-1,/home/edouardfouche/data/subspaces/kdd99-SGMRD-KSP...,113946.9112,0.6998,0.0647,0.1029,0.0,0.002,0.0056,0.0388,0.0500,0.0537,0.0,0.0006,0.0039,0.0545,0.1405,0.2265,SGMRD-MPTS
75,kdd99,LOF-2,/home/edouardfouche/data/subspaces/kdd99-SGMRD-KSP...,117791.6039,0.6173,0.0620,0.0860,0.0,0.000,0.0112,0.0272,0.0524,0.0547,0.0,0.0000,0.0079,0.0382,0.1473,0.2305,SGMRD-MPTS
76,kdd99,LOF-10,/home/edouardfouche/data/subspaces/kdd99-SGMRD-KSP...,119156.5024,0.5120,0.0570,0.0674,0.0,0.002,0.0064,0.0112,0.0156,0.0161,0.0,0.0006,0.0045,0.0157,0.0438,0.0680,SGMRD-MPTS
77,kdd99,LOF-5,/home/edouardfouche/data/subspaces/kdd99-SGMRD-KSP...,114130.4156,0.5486,0.0580,0.0729,0.0,0.004,0.0184,0.0212,0.0300,0.0287,0.0,0.0011,0.0129,0.0298,0.0843,0.1209,SGMRD-MPTS
78,kdd99,LOF-20,/home/edouardfouche/data/subspaces/kdd99-SGMRD-KSP...,130996.6903,0.4479,0.0557,0.0601,0.0,0.000,0.0008,0.0068,0.0076,0.0084,0.0,0.0000,0.0006,0.0096,0.0214,0.0354,SGMRD-MPTS
79,kdd99,LOF-50,/home/edouardfouche/data/subspaces/kdd99-SGMRD-KSP...,127057.1096,0.3157,0.0518,0.0488,0.0,0.000,0.0000,0.0004,0.0002,0.0007,0.0,0.0000,0.0000,0.0006,0.0006,0.0028,SGMRD-MPTS
82,kdd99,LOF-100,/home/edouardfouche/data/subspaces/kdd99-SGMRD-KSP...,144093.4539,0.2259,0.0471,0.0432,0.0,0.000,0.0000,0.0004,0.0072,0.0123,0.0,0.0000,0.0000,0.0006,0.0202,0.0517,SGMRD-MPTS


In [42]:
summaryfull[(summaryfull["refId"] == "kdd99") & (summaryfull["minerId"] == "Hash")]

,refId,minerId,subspaces,minetime,rocauc,prauc,ap,p1,p2,p5,p10,p20,p30,r1,r2,r5,r10,r20,r30,searchers
88,kdd99,Hash,Full,1949.6346,0.2658,0.0461,0.045,0.024,0.024,0.0144,0.0072,0.0036,0.0027,0.0034,0.0067,0.0101,0.0101,0.0101,0.0112,Full


In [43]:
summary[(summary["refId"] == "kdd99") & (summary["minerId"] == "Hash")][interesting] * 100

,minetime,rocauc,ap,p1,p2,p5,r1,r2,r5
48,2973735.13,53.29,7.03,0.4,0.2,0.24,0.06,0.06,0.17
58,2994414.32,44.19,5.95,0.0,0.4,0.16,0.00,0.11,0.11
60,3037476.81,43.21,5.73,0.0,0.0,0.08,0.00,0.00,0.06


In [44]:

a = summaryloffull.loc[summaryloffull.groupby(["refId", "searchers"])["rocauc"].idxmax()]
b = summarylof.loc[summarylof.groupby(["refId", "searchers"])["rocauc"].idxmax()]
c = summaryhash[[x == "Full" for x in summaryhash["searchers"]]]
pd.concat([a,b,c]).groupby(["refId", "minerId", "searchers"])[interesting].mean()*100

minetime  rocauc     ap      p1     p2  \
refId      minerId searchers                                               
activity   Hash    Full          227695.12   95.95  68.23   71.62  72.58   
           LOF-10  GMD-Batch   16030677.02   96.79  81.20   97.75  96.18   
           LOF-5   GMD-Init    17968546.63   97.23  86.56  100.00  98.65   
                   SGMRD-MPTS  16896479.74   97.32  85.39   94.59  94.83   
           LOF-50  Full         4212001.29   93.93  61.80   74.32  64.72   
example_10 Hash    Full           44686.34   71.23   1.87    0.00   0.00   
           LOF-100 GMD-Batch    1361742.37   90.57  55.66   45.00  25.00   
                   GMD-Init     2635870.80   89.43  51.85   44.00  24.50   
                   SGMRD-MPTS   1424496.22   92.70  59.93   50.00  26.00   
           LOF-5   Full          675721.34   88.77  31.44   33.00  18.50   
example_20 Hash    Full           59970.21   48.39   0.80    0.00   0.00   
           LOF-100 GMD-Batch    3261954.86   83.93  39.07   35.00  20.00   
                   SGMRD-MPTS   3441590.82   85.05  41.19   36.00  19.50   
           LOF-2   Full         1367652.88   72.55   5.57    8.00   6.00   
           LOF-5   GMD-Init     3738080.91   78.40  27.91   24.00  14.00   
example_50 Hash    Full          103618.53   46.52   0.73    0.00   0.00   
           LOF-1   Full         2824681.33   61.38   1.08    0.00   0.50   
           LOF-5   GMD-Batch    9691720.29   78.31  30.90   27.00  15.50   
                   SGMRD-MPTS  11152072.57   75.87  31.27   27.00  16.00   
           LOF-50  GMD-Init    22845428.92   62.52   7.28   10.00   6.00   
kdd99      Hash    Full          194963.46   26.58   4.50    2.40   2.40   
           LOF-1   Full         1578129.80   70.37  10.79    0.00   0.00   
                   GMD-Batch   12104868.47   68.74  10.01    0.00   0.20   
                   GMD-Init    12611218.13   73.31  11.38    0.00   0.80   
                   SGMRD-MPTS  11394691.12   69.98  10.29    0.00   0.20   

                                  p5     r1     r2     r5  
refId      minerId searchers                               
activity   Hash    Full        75.00   7.15  14.52  37.48  
           LOF-10  GMD-Batch   88.49   9.75  19.24  44.22  
           LOF-5   GMD-Init    94.87   9.98  19.73  47.42  
                   SGMRD-MPTS  94.24   9.44  18.97  47.10  
           LOF-50  Full        64.03   7.42  12.94  32.00  
example_10 Hash    Full         2.80   0.00   0.00  16.28  
           LOF-100 GMD-Batch   12.00  52.33  58.14  69.77  
                   GMD-Init    10.80  51.16  56.98  62.79  
                   SGMRD-MPTS  12.00  58.14  60.47  69.77  
           LOF-5   Full        10.40  38.37  43.02  60.47  
example_20 Hash    Full         0.00   0.00   0.00   0.00  
           LOF-100 GMD-Batch    8.80  39.77  45.45  50.00  
                   SGMRD-MPTS   9.20  40.91  44.32  52.27  
           LOF-2   Full         4.40   9.09  13.64  25.00  
           LOF-5   GMD-Init     7.60  27.27  31.82  43.18  
example_50 Hash    Full         0.00   0.00   0.00   0.00  
           LOF-1   Full         0.60   0.00   1.23   3.70  
           LOF-5   GMD-Batch    7.40  33.33  38.27  45.68  
                   SGMRD-MPTS   7.60  33.33  39.51  46.91  
           LOF-50  GMD-Init     3.40  12.35  14.81  20.99  
kdd99      Hash    Full         1.44   0.34   0.67   1.01  
           LOF-1   Full         0.08   0.00   0.00   0.06  
                   GMD-Batch    1.84   0.00   0.06   1.29  
                   GMD-Init     3.84   0.00   0.22   2.70  
                   SGMRD-MPTS   0.56   0.00   0.06   0.39

In [45]:
summaryloffull[summaryloffull["refId"] == "kdd99"].groupby(["refId", "minerId", "subspaces"])[interesting].mean() * 100

minetime  rocauc     ap   p1   p2    p5   r1   r2  \
refId minerId subspaces                                                        
kdd99 LOF-1   Full       1578129.80   70.37  10.79  0.0  0.0  0.08  0.0  0.0   
      LOF-10  Full       1734480.18   49.65   6.58  0.0  0.0  0.00  0.0  0.0   
      LOF-100 Full       2070604.30   30.11   4.71  0.0  0.0  0.00  0.0  0.0   
      LOF-2   Full       1609607.44   65.07   9.57  0.0  0.0  0.08  0.0  0.0   
      LOF-20  Full       1793859.34   45.36   6.03  0.0  0.0  0.00  0.0  0.0   
      LOF-5   Full       1659218.17   52.77   7.15  0.0  0.0  0.08  0.0  0.0   
      LOF-50  Full       1959491.98   40.93   5.46  0.0  0.0  0.00  0.0  0.0   

                           r5  
refId minerId subspaces        
kdd99 LOF-1   Full       0.06  
      LOF-10  Full       0.00  
      LOF-100 Full       0.00  
      LOF-2   Full       0.06  
      LOF-20  Full       0.00  
      LOF-5   Full       0.06  
      LOF-50  Full       0.00